In [ ]:
from my_packages import *

from appgeopy import *

In [ ]:
from collections.abc import Mapping


def merge_dicts(*dicts):
    """Recursively merges multiple dictionaries."""
    result = {}
    for dictionary in dicts:
        for key, value in dictionary.items():
            if isinstance(value, Mapping) and key in result:
                # If both values are dictionaries, recursively merge them
                result[key] = merge_dicts(result[key], value)
            else:
                # Otherwise, just set the value from the current dictionary
                result[key] = value
    return result

In [ ]:
folder = r"peakstroughs_Final/Peaks_Troughs/"
excel_files = glob(os.path.join(folder, "*.xlsx"))
excel_files[:5]

In [ ]:
def transform_to_dict(excel_file, extreme_type):
    # Validate extreme_type
    if extreme_type not in ["peaks", "troughs"]:
        raise ValueError("extreme_type must be either 'peaks' or 'troughs'")

    # Proceed with the transformation
    df = pd.read_excel(excel_file, sheet_name=extreme_type)
    dates = df["time"].apply(lambda x: x.strftime("%Y%m%d"))
    dates = dates.to_numpy()
    values = df["value"].to_numpy()

    return {extreme_type: {"date": dates, "value": values}}

In [ ]:
today_string = datetime.now().strftime("%Y/%m/%d, %H:%M:%S")

final_dict_list = []

final_metadata_list = []

for select_file in tqdm(excel_files[:]):
    # ____________________________________________________________________________
    station, wellcode = os.path.basename(select_file).split(".")[0].split("_")

    station_measurement_dict = {station: {wellcode: {}}}

    for extreme_type in ["peaks", "troughs"]:
        extreme_dict = transform_to_dict(select_file, extreme_type)
        station_measurement_dict[station][wellcode].update(extreme_dict)
    # ____________________________________________________________________________
    station_metadata_dict = {station: {"UpdatedTime": f"{today_string}"}}
    # ____________________________________________________________________________
    final_dict_list.append(station_measurement_dict)
    final_metadata_list.append(station_metadata_dict)

In [ ]:
# load old HDF5 data

gwl_hdf5_file = "20240903_GWL_CRFP.h5"

# Extract existing data and metadata
with h5py.File(gwl_hdf5_file, "r") as hdf5_file:
    existing_data_dict = gwatertools.h5pytools.hdf5_to_data_dict(hdf5_file)
    existing_metadata_dict = gwatertools.h5pytools.hdf5_to_metadata_dict(hdf5_file)

In [ ]:
update_info_metadata_dict = {
    "Update004": "2024/09/19 : Add peaks/troughs extracted from modeled GWL using multiple filters. The complete scripts and supporting tools are shown in folder 20240916_Peaks_Troughs_GWL_2"
}
final_metadata_list.append(update_info_metadata_dict)
# ____________________________________________________________________________
new_data_dict = merge_dicts(*final_dict_list)
new_metadata_dict = merge_dicts(*final_metadata_list)

In [ ]:
# Update dictionaries
updated_data_dict = gwatertools.h5pytools.update_data_dict(existing_data_dict, new_data_dict)
updated_metadata_dict = gwatertools.h5pytools.update_metadata_dict(existing_metadata_dict, new_metadata_dict)